# **Análise Exploratória dos Hábitos de Jogo dos Jogadores de Valorant**

## Configurações Iniciais

### Imports

In [17]:
import os
import sys
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, col, expr, count, sum, max, udf, dayofweek, date_format, when, mean, median
from pyspark.sql.types import StringType
sys.path.append('../src/')
from aws.aws import Aws
import io
import pandas as pd 
import boto3
from datetime import datetime
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

### Criando a seção Spark

In [3]:
spark = SparkSession.builder.appName("ValorantDataAnalysis").getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/05/28 18:48:30 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
spark.conf.set("spark.hadoop.fs.s3a.impl", "org.apache.hadoop.fs.s3a.S3AFileSystem")
spark.conf.set("spark.hadoop.fs.s3a.access.key", os.getenv('AWS_ACCESS_KEY_ID'))
spark.conf.set("spark.hadoop.fs.s3a.secret.key", os.getenv('AWS_SECRET_ACCESS_KEY'))

### Config pandas.

In [8]:
pd.set_option('display.max_rows', None)
pd.set_option('display.max_rows', None)
# Configurar o formato dos números
pd.set_option('display.float_format', '{:.2f}'.format)

### Instânciando as classe utilizadas.

In [5]:
aws = Aws()

## Carregando os dataframes.

### Utils.

In [6]:
def get_files(bucket_name : str, folder_path : str) -> list:
    """"""
    objects = aws.list_objetcs_s3(bucket_name, folder_path)


    return objects

def concat_files_s3(objects):
    """"""

    json_files = [obj['Key'] for obj in objects]

    for file in json_files:

        response = aws.read_s3_v2(bucket_name='s3-tcc-fia-valorant', folder_path=file)
        json_data = response['Body'].read().decode('utf-8')

    return io.StringIO(json_data)

def read_spark(data_io):
    """"""
    data_io = pd.read_csv(data_io)
    return spark.createDataFrame(data_io)

def create_dataframe(bucket_name : str, folder_path : str):
    """"""
    objects = get_files(bucket_name, folder_path)
    data_io = concat_files_s3(objects)
    df = read_spark(data_io)

    return df

def save_dataframe_csv(bucket_name, folder_path, file_name, data, file_format):
    # Convert DataFrame to CSV string
    csv_buffer = io.StringIO()
    data.toPandas().to_csv(csv_buffer, index=False)

    # Retrieve CSV data from buffer
    csv_buffer_value = csv_buffer.getvalue()

    date = datetime.now().strftime("%Y%m%d_%H%M%S")
    file_name = file_name + '_' + date + file_format
    file_path = folder_path + file_name

    # Write CSV string to S3
    s3 = boto3.resource('s3')

    try:
        s3.Object(bucket_name, file_path).put(Body=csv_buffer_value)
        print(f"Data was written to S3://{bucket_name}/{folder_path}")

    except Exception as e:

        print(f"Error: {e}")
    
        return False



### df_player_book

In [7]:
df_player_book = create_dataframe('s3-tcc-fia-valorant', 'valorant/refined/player-book/')

## EDA

### Filtros

In [9]:
total_matches_before = df_player_book.select('match_id').distinct().count()
df_player_book = df_player_book.dropna()
total_matches_after = df_player_book.select('match_id').distinct().count()
total_matches_removed = total_matches_before - total_matches_after
print(f"Total matches before removing null values: {total_matches_before}")
print(f"Total matches after removing null values: {total_matches_after}")
print(f"Total matches removed: {total_matches_removed}")

23/05/28 18:56:03 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


Total matches before removing null values: 298
Total matches after removing null values: 218
Total matches removed: 80


In [12]:
total_matches_before = df_player_book.select('match_id').distinct().count()
df_player_book = df_player_book.where((col("total_players_match") == 10) & (col("total_players_team") == 5))
total_matches_after = df_player_book.select('match_id').distinct().count()
total_matches_removed = total_matches_before - total_matches_after
print(f"Total matches before removing null values: {total_matches_before}")
print(f"Total matches after removing null values: {total_matches_after}")
print(f"Total matches removed: {total_matches_removed}")

Total matches before removing null values: 218
Total matches after removing null values: 212
Total matches removed: 6


### Partidas

In [13]:
df_player_book_pd = df_player_book.toPandas()

In [ ]:
df_matches_raw_describe = df_player_book_pd.describe()
df_matches_raw_describe_pivot = pd.pivot_table(df_matches_raw_describe, columns=["count", 'mean', 'std', 'min', '25%', '50%', '75%', 'max'])
df_matches_raw_describe_pivot

#### Dias

In [15]:
df_player_book_pd_days = df_player_book_pd.drop_duplicates(subset=['match_id'])

In [19]:
matches_per_day = (
    
                    df_player_book_pd_days.groupby([df_player_book_pd_days.date_match, df_player_book_pd_days.week_day])['match_id']
                    .count()
                    .reset_index(name='count')
                    .reset_index(drop = True)
                    )
fig = px.bar(x = matches_per_day['date_match'], y = matches_per_day['count'], title= 'Numero de partidas por dia.')
fig.show()

In [21]:
weekDayOrdered = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
mean_matches_per_days_week = (
                                matches_per_day.groupby(matches_per_day.week_day)['count']
                                .mean()
                                .reindex(weekDayOrdered) 
                                .reset_index(name = 'mean')
                                .reset_index(drop = True)
                                .round(3)
                                )

fig = px.line(mean_matches_per_days_week, x = 'week_day', y = 'mean', title = 'Média de partidas por dia.')
fig.show()

In [23]:
means_time_matches = df_player_book_pd_days[['match_id', 'week_day', 'playtime_minutes_value']]
means_time_matches = (
                        means_time_matches.groupby('week_day')['playtime_minutes_value']
                        .mean()
                        .reindex(weekDayOrdered)
                        .reset_index(name = 'mean')
                        .reset_index(drop = True)                     
                        .round(2)
)

fig = px.line(means_time_matches, x = 'week_day', y = 'mean', title = 'Média de tempo jogado por dia da semana.')
fig.show()

In [25]:
top_seven_days_matches = (
                            df_player_book_pd_days.groupby(['date_match', 'week_day'])['week_day']
                            .count()
                            .reset_index(name = 'count')
                            .sort_values('count', ascending = False)
                            .reset_index(drop = True)
                            .head(7)
                            )
fig = px.bar(top_seven_days_matches, x='week_day', y='count',
             hover_data=['date_match', 'count'], color='count', height=400, title = 'Top sete dias por quantidade de partidas.')
fig.show()

**Análise Exploratória dos Hábitos de Jogo dos Jogadores de Valorant**

A análise exploratória revelou números significativos sobre os hábitos de jogo dos jogadores de Valorant. Ao longo da semana, os jogadores realizam em média **1,75 a 2,22 jogos por dia**. Essa variação é evidente ao observar as médias de jogos por dia da semana:

**Quartas-feiras, quintas-feiras e sábados** registram cerca de **1,8 jogos**.
**Segundas e sextas-feiras** têm aproximadamente **1,73** a **1,80 jogos**.
As **terças-feiras** apresentam a **média mais baixa**, com **1,75 jogos**.
No entanto, os **domingos se destacam** com a **média mais alta** de **2,22 jogos**.
Além disso, é importante ressaltar alguns dias específicos com números expressivos de jogos. O dia **19/12/2022 (segunda-feira)** e o dia **01/05/2022 (domingo)** registraram um total de **5 jogos cada**. Já os dias **12/05/2023 (sexta-feira) e 06/05/2023 (sábado)** contabilizaram **4 jogos cada**.

Esses números destacados indicam momentos de maior participação dos jogadores, **sugerindo a ocorrência de eventos especiais**, torneios ou outras atividades que estimularam a **intensificação do engajamento dos jogadores**.

Esses insights podem orientar a **alocação de recursos e a implementação de iniciativas** direcionadas para maximizar o engajamento dos jogadores de Valorant.

### Plantio e desarmamento da spike

In [26]:
matches_plat_defuse = (
                        df_player_book_pd_days.groupby(['result'])[['plants_value', 'defuses_value']]
                        .sum()
                        .sort_values('plants_value', ascending = False)
                        .reset_index()
)

fig = px.bar(matches_plat_defuse, x = 'result', y = 'plants_value', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

fig = px.bar(matches_plat_defuse, x = 'result', y = 'defuses_value', text_auto='.2s',
            title="Default: various text sizes, positions and angles", color = 'result')
fig.show()

fig = go.Figure()
fig.add_trace(go.Box(y=matches_plat_defuse['plants_value'],
                    name='plantsValue'))
fig.add_trace(go.Box(y=matches_plat_defuse['defuses_value'],
                    name='defusesValue'))
fig.show()

Em média, **equipes vitoriosas** plantaram a spike **135 vezes** e desarmaram **37 vezes**.
Equipes que **sofreram derrotas** plantaram a spike **130 vezes** e desarmaram **46 vezes**.
**Em empates**, ocorreram apenas **6 plantios da spike** e **nenhum** desarme.
Esses dados destacam a **importância dos plantio e desarmes** para o resultado das partidas de Valorant. Equipes com um **alto número de plantas e desarmes** têm uma **maior probabilidade de alcançar a vitória**. Portanto, é crucial para os jogadores e equipes desenvolverem estratégias eficientes para essas ações dentro do jogo.

### Mapas

In [40]:
maps_matches = df_player_book_pd[['match_id', 'map_name']]
maps_matches = (
                maps_matches.groupby('map_name')['match_id']
                .count()
                .reset_index(name='count')
                .reset_index(drop=True)
                .sort_values('count', ascending = True)

)

fig = px.bar(maps_matches, x='map_name', y='count', title='Numero de partidas por mapa.')
fig.show()


Observa-se que o mapa **Haven possui o maior número de partidas**, com um total de **390**. Em seguida, temos os mapas **Ascent** e **Bind** com **370 e 369 partidas**, respectivamente. Por outro lado, os mapas **Breeze** e **Fracture**, ambos com **180 partidas**, apresentam uma quantidade menor de partidas registradas.

É **importante ressaltar** que a quantidade de partidas pode estar relacionada à **popularidade e tempo de existência dos mapas**. Os mapas **mais antigos**, como **Haven, "Ascent e Bind**, têm mais tempo para acumular partidas em comparação com os mapas **mais recentes**, como **Breeze e Fracture**. Portanto, a distribuição reflete não apenas a preferência dos jogadores, mas também a idade dos mapas, sendo esperado que os mapas mais novos aumentem sua quantidade de partidas ao longo do tempo.

Vale lembrar que em Valorant a **Riot** tenta garantir uma **distribuição igualitaria** entre os mapas.